In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle (1).json to kaggle (1).json


In [3]:
!mkdir -p ~/.kaggle
!mv /content/kaggle\ \(1\).json ~/.kaggle/kaggle.json  # Use the exact filename with escaped space
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!pip install kaggle
!kaggle datasets download -d anshumanprabhakar/depression-detection-using-mri -p /content/data/raw/ --unzip

Dataset URL: https://www.kaggle.com/datasets/anshumanprabhakar/depression-detection-using-mri
License(s): CC0-1.0
 98% 229M/233M [00:00<00:00, 654MB/s] 
100% 233M/233M [00:00<00:00, 684MB/s]


In [6]:
!pip install numpy pandas scikit-learn matplotlib seaborn nibabel nilearn torch torchvision streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.1 MB/s eta 0:00:00


In [12]:
import os
import nibabel as nib
from nilearn import image, datasets, masking
import numpy as np
import pandas as pd

# Set data path
data_dir = '/content/data/raw/'
files = [f for f in os.listdir(data_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]

# Assume labels from filenames (adjust based on actual structure)
labels = []
for f in files:
    if 'depressed' in f.lower():  # Customize parsing (e.g., 'depressed_001.nii')
        labels.append(1)
    else:
        labels.append(0)
print(f"Loaded {len(files)} files. Class distribution: {np.bincount(labels)}")

# If no labels, manually assign based on study (e.g., first 147 depressed, rest healthy)
if len(set(labels)) == 1:  # No variation
    labels = [1] * 147 + [0] * (len(files) - 147)  # Adjust based on split

Loaded 0 files. Class distribution: []


In [17]:
from nilearn.connectome import ConnectivityMeasure
from nilearn.input_data import NiftiLabelsMasker
import os
import nibabel as nib
from nilearn import image, datasets
import numpy as np
import pandas as pd

# Set data path to the subdirectory
data_dir = '/content/data/raw/Functional MRI of emotional memory in adolescent depression/'
files = [f for f in os.listdir(data_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]
print(f"Found files: {files}")  # Debug output
print(f"Number of files: {len(files)}")

if not files:
    raise ValueError("No MRI files found in the specified directory. Please check the path and file extensions.")

# Assume labels from filenames and track indices
labels = []
file_indices = {}  # Map filename to index
for i, f in enumerate(files):
    if 'depressed' in f.lower() or 'dep' in f.lower() or 'unmeddepsubs' in f.lower():  # Broaden parsing
        labels.append(1)
    else:
        labels.append(0)
    file_indices[f] = i
print(f"Class distribution: {np.bincount(labels)}")

# Load Harvard-Oxford atlas
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
masker = NiftiLabelsMasker(labels_img=atlas.maps, strategy='mean', standardize=False)

# Extract features
X = []
failed_files = []
for f in files:
    img_path = os.path.join(data_dir, f)
    try:
        img = nib.load(img_path)
        print(f"Original shape of {f}: {img.shape}")  # Debug shape

        # Resample to atlas space with copy_header
        resampled_img = image.resample_to_img(img, atlas.maps, interpolation='nearest', copy_header=True)
        print(f"Resampled shape of {f}: {resampled_img.shape}")  # Debug resampled shape

        # Check if resampled data is valid
        data = resampled_img.get_fdata()
        if np.all(data == 0):
            print(f"Warning: {f} resampled data is all zeros. Skipping.")
            failed_files.append(f)
            continue

        time_series = masker.fit_transform(resampled_img)
        print(f"Time series shape for {f}: {time_series.shape}")  # Debug time series shape

        # Handle 1D time series (e.g., 48 time points)
        if len(time_series.shape) == 1 and len(time_series) >= 20:
            mean_signal = time_series[:20]  # Take first 20 time points as features
        elif len(time_series.shape) >= 2 and time_series.shape[1] >= 20:
            mean_signal = np.mean(time_series, axis=0)[:20]  # Mean of first 20 ROIs
        else:
            # Pad with zeros if fewer than 20 features
            mean_signal = np.pad(time_series, (0, 20 - len(time_series)), mode='constant')
            print(f"Padding {f} with zeros to reach 20 features.")

        X.append(mean_signal)
    except Exception as e:
        print(f"Failed to process {f}: {e}")
        failed_files.append(f)
        continue

X = np.array(X)
y = np.array(labels)  # Initial labels for all files

# Filter y to match the processed files (remove labels for failed files)
success_indices = [i for i, f in enumerate(files) if f not in failed_files]
y = y[success_indices]

print(f"Number of processed samples: {len(X)}")
print(f"Number of labels: {len(y)}")
print(f"Failed files: {failed_files}")

# Feature names
roi_names = atlas.labels[:20]
df = pd.DataFrame(X, columns=[f'ROI_{i}' for i in range(20)])
df['Depression'] = y  # Should now match lengths

# Create directory if it doesn't exist
os.makedirs('/content/data/processed/', exist_ok=True)
df.to_csv('/content/data/processed/processed_mri_data.csv', index=False)
print(df.head())

Found files: ['Encode.Neg_gt_Pos.Hits.UnMedDepSubs.GroupMean.nii', 'Retrieve.CR_gt_Baseline.NegWords.UnMedDepSubs.Corr_Age.nii', 'Retrieve.Hit_gt_CR.PosWords.DepSubs.GroupMean.nii', 'Retrieve.Neg_gt_Pos.AllTrials.DepSubs.Corr_SMFQ.nii', 'Retrieve.Trial_gt_Baseline.AllWords.DepSubs.GroupMean.nii', 'Retrieve.Hit_gt_CR.AllWords.AllSubs.Dep_gt_Con.nii', 'Retrieve.Hit_gt_CR.AllWords.AllSubs.GroupMean.nii', 'Encode.Hit_gt_Miss.AllWords.UnMedSubs.DepCon_int_Age.nii', 'Encode.Neg_gt_Pos.AllTrials.DepSubs.GroupMean.nii', 'Encode.Hit_gt_Baseline.NegWords.UnMedSubs.GroupMean.nii', 'Encode.Trial_gt_Baseline.AllWords.UnMedDepSubs.Corr_SMFQ.nii', 'Encode.Trial_gt_Baseline.AllWords.DepSubs.GroupMean.nii', 'Retrieve.CR_gt_Baseline.NegWords.AllSubs.Dep_gt_Con.nii', 'Encode.Trial_gt_Baseline.AllWords.AllSubs.Corr_Age.nii', 'Encode.Neg_gt_Pos.Hits.DepSubs.GroupMean.nii', 'Retrieve.Trial_gt_Baseline.AllWords.UnMedSubs.Corr_Age.nii', 'Encode.Hit_gt_Miss.AllWords.UnMedDepSubs.GroupMean.nii', 'Retrieve.Neg_g

[fetch_atlas_harvard_oxford] Dataset found in /root/nilearn_data/fsl

Original shape of Encode.Neg_gt_Pos.Hits.UnMedDepSubs.GroupMean.nii: (91, 109, 91)
Resampled shape of Encode.Neg_gt_Pos.Hits.UnMedDepSubs.GroupMean.nii: (91, 109, 91)


/tmp/ipython-input-3183142286.py:43: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_img = image.resample_to_img(img, atlas.maps, interpolation='nearest', copy_header=True)


Time series shape for Encode.Neg_gt_Pos.Hits.UnMedDepSubs.GroupMean.nii: (48,)
Original shape of Retrieve.CR_gt_Baseline.NegWords.UnMedDepSubs.Corr_Age.nii: (91, 109, 91)
Resampled shape of Retrieve.CR_gt_Baseline.NegWords.UnMedDepSubs.Corr_Age.nii: (91, 109, 91)


...

Time series shape for Encode.Hit_gt_Baseline.NegWords.UnMedSubs.DepCon_int_Age.nii: (48,)
Original shape of Retrieve.CR_gt_Baseline.NegWords.UnMedSubs.GroupMean.nii: (91, 109, 91)
Resampled shape of Retrieve.CR_gt_Baseline.NegWords.UnMedSubs.GroupMean.nii: (91, 109, 91)


/tmp/ipython-input-3183142286.py:43: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_img = image.resample_to_img(img, atlas.maps, interpolation='nearest', copy_header=True)


Time series shape for Retrieve.CR_gt_Baseline.NegWords.UnMedSubs.GroupMean.nii: (48,)
Original shape of Encode.Neg_gt_Pos.AllTrials.UnMedDepSubs.GroupMean.nii: (91, 109, 91)
Resampled shape of Encode.Neg_gt_Pos.AllTrials.UnMedDepSubs.GroupMean.nii: (91, 109, 91)


/tmp/ipython-input-3183142286.py:43: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_img = image.resample_to_img(img, atlas.maps, interpolation='nearest', copy_header=True)


Time series shape for Encode.Neg_gt_Pos.AllTrials.UnMedDepSubs.GroupMean.nii: (48,)
Number of processed samples: 280
Number of labels: 280
Failed files: ['Encode.Miss_gt_Baseline.PosWords.UnMedDepSubs.Corr_Age.nii', 'Encode.Miss_gt_Baseline.PosWords.AllSubs.DepCon_int_Age.nii', 'Encode.Miss_gt_Baseline.PosWords.DepSubs.GroupMean.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedDepSubs.Corr_SMFQ.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedDepSubs.GroupMean.nii', 'Encode.Miss_gt_Baseline.PosWords.AllSubs.Corr_Age.nii', 'Encode.Miss_gt_Baseline.PosWords.DepSubs.Corr_SMFQ.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedSubs.GroupMean.nii', 'Encode.Miss_gt_Baseline.PosWords.AllSubs.GroupMean.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedSubs.Corr_Age.nii', 'Encode.Miss_gt_Baseline.PosWords.DepSubs.Corr_Age.nii', 'Encode.Miss_gt_Baseline.PosWords.AllSubs.Dep_gt_Con.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedSubs.Dep_gt_Con.nii', 'Encode.Miss_gt_Baseline.PosWords.UnMedSubs.DepCon_int_Age.nii']
   

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np
import joblib
import os

# Load the processed data
df = pd.read_csv('/content/data/processed/processed_mri_data.csv')

# Prepare features and target
X = df.drop('Depression', axis=1).values
y = df['Depression'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Feature selection (top 15 features)
selector = SelectKBest(mutual_info_classif, k=15)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Train models
lr_model = LogisticRegression().fit(X_train_selected, y_train)
dt_model = DecisionTreeClassifier(max_depth=3).fit(X_train_selected, y_train)  # Shallow for interpretability
svm_model = SVC(kernel='linear', probability=True).fit(X_train_selected, y_train)

# Save models and scaler
os.makedirs('/content/model/', exist_ok=True)
joblib.dump(lr_model, '/content/model/lr_model.pkl')
joblib.dump(scaler, '/content/model/scaler.pkl')
joblib.dump(selector, '/content/model/selector.pkl')

# Interpretability: Feature importance for Logistic Regression
selected_features = df.drop('Depression', axis=1).columns[selector.get_support()].tolist()
lr_coefficients = pd.DataFrame({'Feature': selected_features, 'Coefficient': lr_model.coef_[0]})
print("Logistic Regression Coefficients:")
print(lr_coefficients.sort_values(by='Coefficient', key=abs, ascending=False))

# Visualize Decision Tree
plt.figure(figsize=(20, 10))
plot_tree(dt_model, feature_names=selected_features, class_names=['Healthy', 'Depressed'], filled=True)
os.makedirs('/content/figures/', exist_ok=True)  # Create figures directory
plt.savefig('/content/figures/decision_tree.png')
plt.close()

print("Models trained and saved successfully!")

Logistic Regression Coefficients:
   Feature  Coefficient
12  ROI_17    -1.037517
6    ROI_7    -0.953042
7   ROI_10     0.896622
3    ROI_4    -0.824381
14  ROI_19     0.784224
0    ROI_0     0.666943
4    ROI_5     0.411250
8   ROI_11    -0.355296
9   ROI_12     0.329387
1    ROI_1     0.270924
13  ROI_18    -0.262964
10  ROI_15    -0.251702
5    ROI_6    -0.138286
11  ROI_16     0.032697
2    ROI_2    -0.016167
Models trained and saved successfully!


In [20]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix
import seaborn as sns

# Predictions and metrics for Logistic Regression
y_pred_lr = lr_model.predict(X_test_selected)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred_lr):.2f}")
print(f"Logistic Regression AUC: {roc_auc_score(y_test, lr_model.predict_proba(X_test_selected)[:, 1]):.2f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_lr)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Depressed'], yticklabels=['Healthy', 'Depressed'])
plt.title('Confusion Matrix - Logistic Regression')
plt.savefig('/content/figures/confusion_matrix.png')
plt.close()

# 5-fold Cross-Validation
cv_scores = cross_val_score(lr_model, X_train_selected, y_train, cv=5, scoring='roc_auc')
print(f"Logistic Regression 5-fold CV AUC: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

# ROC Curve (optional, for deeper analysis)
fpr, tpr, _ = roc_curve(y_test, lr_model.predict_proba(X_test_selected)[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc_score(y_test, lr_model.predict_proba(X_test_selected)[:, 1]):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend(loc='best')
plt.savefig('/content/figures/roc_curve.png')
plt.close()

print("Evaluation completed and visualizations saved!")

Logistic Regression Accuracy: 0.70
Logistic Regression AUC: 0.69
Logistic Regression 5-fold CV AUC: 0.69 ± 0.04
Evaluation completed and visualizations saved!


In [22]:
%%writefile ui.py

Writing ui.py


In [28]:
!streamlit run ui.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.53.61.119:8501

⠇⠏⠋⠙⠹⠸your url is: https://floppy-crabs-create.loca.lt
/content/ui.py:3: DeprecationWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  from nilearn.input_data import NiftiLabelsMasker
[fetch_atlas_harvard_oxford] Dataset found in /root/nilearn_data/fsl
[fetch_atlas_harvard_oxford] Dataset found in /root/nilearn_data/fsl
  Stopping...
^C


In [27]:
!curl ipv4.icanhazip.com

34.53.61.119


In [34]:
# Install required packages
!pip install streamlit -q
!pip install pyngrok -q

# Set up ngrok (one-time setup)
import getpass
import os
from pyngrok import ngrok

# Get ngrok auth token (run once)
if not os.path.exists("/content/ngrok_auth.txt"):
    print("Please enter your ngrok auth token (get it from https://dashboard.ngrok.com/auth):")
    auth_token = getpass.getpass()
    with open("/content/ngrok_auth.txt", "w") as f:
        f.write(auth_token)
    # Set the auth token
    !ngrok authtoken $(cat /content/ngrok_auth.txt)
else:
    # Use existing auth token
    !ngrok authtoken $(cat /content/ngrok_auth.txt)

# Run Streamlit and tunnel with ngrok
!streamlit run ui.py &>/content/logs.txt &  # Redirect output to log file
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Keep the notebook running (optional: add a sleep to prevent immediate termination)
import time
time.sleep(3600)  # Run for 1 hour (adjust as needed)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://polyarchic-nonintrospectively-felisha.ngrok-free.dev" -> "http://localhost:8501"


KeyboardInterrupt: 